In [1]:
# Cell 1: Setup
import torch
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Cell 2: Image Load and Preprocess
def load_image(img_path, max_size=128):
    image = Image.open(img_path).convert("RGB")
    in_transform = transforms.Compose([
        transforms.Resize((max_size, max_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])
    image = in_transform(image).unsqueeze(0)
    return image.to(device)


In [3]:
# Cell 3: Utility Functions
def gram_matrix(tensor):
    _, d, h, w = tensor.size()
    tensor = tensor.view(d, h * w)
    return torch.mm(tensor, tensor.t())

def get_features(image, model):
    layers = {'0': 'conv1_1', '5': 'conv2_1', '10': 'conv3_1', '19': 'conv4_1', '21': 'conv4_2'}
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
    return features

def im_convert(tensor):
    image = tensor.to("cpu").clone().detach().squeeze(0)
    image = image * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    image = image + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    image = image.clamp(0, 1)
    return transforms.ToPILImage()(image)

In [5]:
# Cell 4: Load content and style image
content = load_image("C:/Users/DELL/Downloads/Neural_Style_Transfer_CodeTech/content.jpg")   # <-- Replace with your image path
style = load_image("C:/Users/DELL/Downloads/Neural_Style_Transfer_CodeTech/style.jpg")

In [6]:
# Cell 5: Run Fast Style Transfer
vgg = models.vgg19(pretrained=True).features.to(device).eval()

for param in vgg.parameters():
    param.requires_grad = False

content_features = get_features(content, vgg)
style_features = get_features(style, vgg)
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

target = content.clone().requires_grad_(True)
optimizer = optim.Adam([target], lr=0.01)

steps = 30
style_weight = 1e5
content_weight = 1

for step in range(steps):
    target_features = get_features(target, vgg)
    content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2'])**2)

    style_loss = 0
    for layer in style_grams:
        target_gram = gram_matrix(target_features[layer])
        style_gram = style_grams[layer]
        layer_loss = torch.mean((target_gram - style_gram)**2)
        style_loss += layer_loss

    total_loss = content_weight * content_loss + style_weight * style_loss

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step % 10 == 0:
        print(f"Step {step}, Loss: {total_loss.item():.4f}")

C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Step 0, Loss: 3578658029568.0000
Step 10, Loss: 3048254996480.0000
Step 20, Loss: 2699542921216.0000


In [8]:
# Cell 6: Save or show output
final_img = im_convert(target)
final_img.save("output.jpg")
final_img.show()